# Creating a basic bot using PyTorch

# 1)- Importing key Modules

In [1]:
import numpy as np
import random
import json

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from nltk_utils import bag_of_words, tokenize, stem
from model import NeuralNet

# 2-Load Data

In [2]:
with open('intents.json', 'r') as f:
    data = json.load(f)

In [3]:
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'How are you',
    'Is anyone there?',
    'Hello',
    'Good day',
    'Whats up'],
   'responses': ['Hello!',
    'Good to see you again!',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['cya',
    'See you later',
    'Goodbye',
    'I am Leaving',
    'Have a Good day'],
   'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'],
   'context_set': ''},
  {'tag': 'age',
   'patterns': ['how old',
    'how old is Hassan',
    'what is your age',
    'how old are you',
    'age please?'],
   'responses': ['I am 32 years old!', '32 years young!'],
   'context_set': ''},
  {'tag': 'name',
   'patterns': ['what is your name',
    'what should I call you',
    'whats your name?'],
   'responses': ['You can call me Hassan.',
    "I'm Hassan!",
    "I'm Hassan Sherwani.You may call me Hassan."],
   'context_set': ''},
  {'tag': 'profession',
   'patterns': ['Id like to ge

In [4]:
data['intents']

[{'tag': 'greeting',
  'patterns': ['Hi',
   'How are you',
   'Is anyone there?',
   'Hello',
   'Good day',
   'Whats up'],
  'responses': ['Hello!',
   'Good to see you again!',
   'Hi there, how can I help?'],
  'context_set': ''},
 {'tag': 'goodbye',
  'patterns': ['cya',
   'See you later',
   'Goodbye',
   'I am Leaving',
   'Have a Good day'],
  'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'],
  'context_set': ''},
 {'tag': 'age',
  'patterns': ['how old',
   'how old is Hassan',
   'what is your age',
   'how old are you',
   'age please?'],
  'responses': ['I am 32 years old!', '32 years young!'],
  'context_set': ''},
 {'tag': 'name',
  'patterns': ['what is your name',
   'what should I call you',
   'whats your name?'],
  'responses': ['You can call me Hassan.',
   "I'm Hassan!",
   "I'm Hassan Sherwani.You may call me Hassan."],
  'context_set': ''},
 {'tag': 'profession',
  'patterns': ['Id like to get some information about you',
   'what is your 

In [5]:
data['intents']

[{'tag': 'greeting',
  'patterns': ['Hi',
   'How are you',
   'Is anyone there?',
   'Hello',
   'Good day',
   'Whats up'],
  'responses': ['Hello!',
   'Good to see you again!',
   'Hi there, how can I help?'],
  'context_set': ''},
 {'tag': 'goodbye',
  'patterns': ['cya',
   'See you later',
   'Goodbye',
   'I am Leaving',
   'Have a Good day'],
  'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'],
  'context_set': ''},
 {'tag': 'age',
  'patterns': ['how old',
   'how old is Hassan',
   'what is your age',
   'how old are you',
   'age please?'],
  'responses': ['I am 32 years old!', '32 years young!'],
  'context_set': ''},
 {'tag': 'name',
  'patterns': ['what is your name',
   'what should I call you',
   'whats your name?'],
  'responses': ['You can call me Hassan.',
   "I'm Hassan!",
   "I'm Hassan Sherwani.You may call me Hassan."],
  'context_set': ''},
 {'tag': 'profession',
  'patterns': ['Id like to get some information about you',
   'what is your 

# 3)- Exploring data

In [6]:
all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in data['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w) # notice use of extend instead of append
        # add to xy pair
        xy.append((w, tag))

print(all_words)


['Hi', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hello', 'Good', 'day', 'Whats', 'up', 'cya', 'See', 'you', 'later', 'Goodbye', 'I', 'am', 'Leaving', 'Have', 'a', 'Good', 'day', 'how', 'old', 'how', 'old', 'is', 'Hassan', 'what', 'is', 'your', 'age', 'how', 'old', 'are', 'you', 'age', 'please', '?', 'what', 'is', 'your', 'name', 'what', 'should', 'I', 'call', 'you', 'whats', 'your', 'name', '?', 'Id', 'like', 'to', 'get', 'some', 'information', 'about', 'you', 'what', 'is', 'your', 'profession', 'what', 'do', 'you', 'do', 'for', 'work', '?', 'Could', 'I', 'know', 'what', 'is', 'your', 'field', '?', 'when', 'are', 'you', 'available', 'for', 'discussion', '?', 'what', 'are', 'your', 'office', 'hours', 'hours', 'of', 'work']


This is like our vocab.

In [7]:
# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort. Set does trick of getting rid of all duplicated ones
all_words = sorted(set(all_words))
tags = sorted(set(tags))

In [8]:
print(len(xy), "patterns")

26 patterns


In [9]:
print(len(tags), "tags:", tags)

6 tags: ['age', 'goodbye', 'greeting', 'hours', 'name', 'profession']


In [10]:
print(len(all_words), "unique stemmed words:", all_words)

49 unique stemmed words: ['a', 'about', 'age', 'am', 'anyon', 'are', 'avail', 'call', 'could', 'cya', 'day', 'discuss', 'do', 'field', 'for', 'get', 'good', 'goodby', 'hassan', 'have', 'hello', 'hi', 'hour', 'how', 'i', 'id', 'inform', 'is', 'know', 'later', 'leav', 'like', 'name', 'of', 'offic', 'old', 'pleas', 'profess', 'see', 'should', 'some', 'there', 'to', 'up', 'what', 'when', 'work', 'you', 'your']


# 4)-Train-Test Split

In [11]:
# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

In [12]:
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [13]:
y_train

array([2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 4, 4, 4, 5, 5, 5,
       5, 3, 3, 3])

In [14]:
print(X_train.shape)
print(y_train.shape)

(26, 49)
(26,)


### Hyper-parameters defined

In [15]:
# Hyper-parameters 
num_epochs = 1200
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size) 
print(output_size)

49
6


- Input beig X and its feature 

- y being number of classes we have

# 5)- Model Building

In [16]:
class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)


### Loss and optimizer

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### Train the model

In [18]:
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(device=device,dtype=torch.int64)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward for back-propagation and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


print(f'final loss: {loss.item():.4f}')

Epoch [100/1200], Loss: 1.8693
Epoch [200/1200], Loss: 0.3177
Epoch [300/1200], Loss: 0.0538
Epoch [400/1200], Loss: 0.0278
Epoch [500/1200], Loss: 0.0008
Epoch [600/1200], Loss: 0.0056
Epoch [700/1200], Loss: 0.0033
Epoch [800/1200], Loss: 0.0015
Epoch [900/1200], Loss: 0.0015
Epoch [1000/1200], Loss: 0.0007
Epoch [1100/1200], Loss: 0.0004
Epoch [1200/1200], Loss: 0.0001
final loss: 0.0001


### save model

In [19]:
data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

training complete. file saved to data.pth


# END OF NOTEBOOK